In [1]:
import torch
import torch.nn as nn
import PIL.Image
import io
import numpy as np
import datasets
import torchaudio
from IPython.display import Audio as play
from torchvision.transforms.v2 import CenterCrop, PILToTensor, ToPILImage, Pad, CenterCrop
from types import SimpleNamespace
from autocodec.codec import AutoCodecND, latent_to_pil, pil_to_latent

In [2]:
device = "cpu"
valid_dataset = dataset = datasets.load_dataset("danjacobellis/musdb_segments",split='validation')

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

In [3]:
checkpoint = torch.load('checkpoint.pth', map_location=device,weights_only=False)
config = checkpoint['config']
model = AutoCodecND(
    dim=1,
    input_channels=config.input_channels,
    J=int(np.log2(config.F)),
    latent_dim=config.latent_dim,
    lightweight_encode=config.lightweight_encode,
    lightweight_decode=config.lightweight_decode,
    post_filter=config.post_filter
).to(device)

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [4]:
L = 2**19
C = config.input_channels
center_crop = CenterCrop((1, L))
def valid_collate_fn(batch):
    B = len(batch)
    x = torch.zeros((B, C, L), dtype=torch.float)
    for i_sample, sample in enumerate(batch):
        xi = torch.zeros((C, 1, 2**21), dtype=torch.int16)
        audio_mix, fs = torchaudio.load(sample['audio_mix']['bytes'], normalize=False)
        xi[:, 0, :] = audio_mix
        xi = center_crop(xi).to(torch.float)
        xi = xi.squeeze(1)
        xi = xi - xi.mean()
        max_abs = xi.abs().max()
        xi = xi / (max_abs + 1e-8)
        x[i_sample, :, :] = xi
    return x

In [5]:
batch = dataset.select(range(4))
x = valid_collate_fn(batch).to(device)
model.eval()
with torch.no_grad():
    z = model.encode(x)
    latent = model.quantize.compand(z)
    x_hat = model.decode(latent)
    if config.post_filter:
        x_hat = model.post_filter(x_hat)
    x_hat = x_hat.clamp(-1,1)
mse = torch.nn.functional.mse_loss(x,x_hat)
PSNR = -10*mse.log10().item() + 6.02
print(f"{PSNR} dB PSNR")

43.85758640289306 dB PSNR


In [ ]:
play(x[0].cpu(),rate=44100)

In [ ]:
play(x_hat[0].cpu(),rate=44100)